In [1]:
import numpy as np
import pandas as pd
import functions.multi_occupation_network as multi_occupation_network

# Calibration Excersize 1: Single Occupation

Suppose there is just one type of labor, in other words that $\mathcal{O}=1$. We calibrate our model to mathc BEA input-output tables, and treat the residual, the part of production not accounted for by intermediate inputs, as the labor elasticity. 

In [2]:
data_dir = '../data/clean/'
dfA      = pd.read_csv(data_dir + 'A.csv')
dfParam  = pd.read_csv(data_dir + 'params.csv')
dfLshare = pd.read_csv(data_dir + 'labor_share.csv')
dfLabor_market_yearly= pd.read_csv(data_dir + 'labor_market_yearly.csv')
dfLabor_market_yearly = dfLabor_market_yearly.sort_values(by=['year', 'BEA_sector'])
dfLabor_market_yearly = dfLabor_market_yearly.dropna(axis=0)
dfLabor_market_yearly = dfLabor_market_yearly[dfLabor_market_yearly['year'] == 2005]
# reformatting parameters
Omega = np.array(dfA.iloc[:, 1:], dtype='float64')
Psi = np.linalg.inv(np.eye(Omega.shape[0])-Omega)
J = Omega.shape[0]
O = 1

epsN = np.array(dfParam.α).reshape((J,O))
epsD = np.array(dfParam.θ_alt).reshape((J,1))

# labor market stats
θ = dfLabor_market_yearly['v'].sum() / dfLabor_market_yearly['u'].sum() 
ν = np.ones(O) * 0.5
curlyQ = np.diag(-ν)
curlyF =  np.eye(O) + curlyQ
r = 0.0282
curlyT = np.diag(r / (θ**(-ν) - r))
# NOTE: here I am picking the r so the τ's roughly look right. Down the line, we can calibrate τ directly from the data. 
# Also note that tightness is computed from unemployed workers, but in our model it should be computed from the whole work force.

curlyL = np.array(dfLabor_market_yearly['e']/dfLabor_market_yearly['e'].sum()).reshape((O,J))


First, we define the shocks we are interested in to allow us to easily change these in the future, that is we define the vectors $d\log\bm{A}$ and $d\log\bm{H}$.

In [3]:
dlog_A = np.zeros((J, 1))
dlog_A[3] = 0.01 
dlog_H = np.zeros((O,1))

We assume Cobb-Douglas production and preferences.

In [4]:
dlog_lam = np.zeros_like(dlog_A)
dlog_epsN = np.zeros_like(epsN)
dlog_epsD = np.zeros_like(epsD)

Finally, we need to make some assumption about how wages change. We begin by assuming that nominal wages do not change in respond to changes in either productivity or the labor force.

In [5]:
epsW_A = np.zeros((O,J))
epsW_H = np.zeros((O,O))
dlog_w = multi_occupation_network.WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)

With these pieces in hand, we can now estimate responses to tightness. For details, see the multiple occupations example code.

In [6]:
curlyE = multi_occupation_network.curlyEFunc(dlog_epsN,epsN)
num=1
dlog_theta = multi_occupation_network.ThetaFunc(dlog_A, dlog_H, dlog_w, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN, num=num)
dlog_theta

array([[0.16355163]])

In [7]:
dlog_p = multi_occupation_network.PriceFunc(dlog_A, dlog_w, dlog_theta, Psi, curlyQ, epsN, curlyT)
dlog_p

array([[-1.51291126e-05],
       [-9.14698005e-15],
       [-6.35340118e-03],
       [-1.96007382e-02],
       [ 2.27184233e-04],
       [ 8.51428657e-04],
       [-1.33957531e-04],
       [-1.00139535e-04],
       [-1.13561706e-03],
       [-2.79704513e-03],
       [-2.62295650e-03],
       [-9.45934332e-04],
       [ 1.54781445e-04],
       [-1.11749911e-03],
       [ 2.45261232e-04],
       [-6.34728060e-04],
       [ 3.83022630e-04]])

In [8]:
dlog_y = multi_occupation_network.OutputFunc(dlog_A,dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE)
dlog_y

array([[0.08179094],
       [0.08177582],
       [0.08812922],
       [0.10137655],
       [0.08154863],
       [0.08092439],
       [0.08190977],
       [0.08187596],
       [0.08291143],
       [0.08457286],
       [0.08439877],
       [0.08272175],
       [0.08162103],
       [0.08289331],
       [0.08153055],
       [0.08241054],
       [0.08139279]])

In [9]:
dlog_aggY = multi_occupation_network.AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)
dlog_aggY

array([[0.08706747]])

In [11]:
epsD.T @ Psi @ dlog_A

array([[0.00717491]])